# Benchmarks

Ji, Shaolin, Shige Peng, Ying Peng, and Xichuan Zhang. “Three Algorithms for Solving High-Dimensional Fully-Coupled FBSDEs through Deep Learning.” ArXiv:1907.05327 [Cs, Math], February 2, 2020. http://arxiv.org/abs/1907.05327.

# Setup

Consider

$$
\newcommand{\R}{\mathbb R}
W \in \R ^m, N \in \R^\ell\\
X\in \R^d, Y\in \R^n, Z\in \R^{n\times m}, R \in \R ^{n\times \ell}\\
\gamma \in \R^{d \times \ell}, \sigma \in \R^{d\times m} \\
f\in C^2(\R^d, E\subset\R^n)\\
$$

For the control problem with jumps, we would have the FBSDE of the form (assuming zero drift on $X$)

$$
\begin{aligned}
dX_t &= \sigma_t dW_t + \gamma_t dN_t\\
dY_t &= (\dots)dt + Z_t dW_t + R_t dN_t
\end{aligned}
$$

Assume that $Y=f(X)$. Necessarily, by Ito's lemma (Oksendal, p.9, 1.2.8; for k-th column):

$$
\begin{aligned}
Z_t &= (D f(X_t))^T \cdot \sigma_t \\
R^{(\cdot k)}_t &= f\left(X_t+\gamma ^{(\cdot k)}_t\right) - f(X_t)
\end{aligned}
$$

Assuming that $D f(X) D f(X)^T$ is never singular and $f^{-1}\in C^2(E, \R^d)$ exists, we have

$$
\begin{aligned}
\sigma_t &= \left(D f(X_t) D f(X_t) ^T \right) ^{-1} D f(X_t) Z =: \beta(X_t) Z_t\\
\gamma^{(\cdot k)}_t &= f^{-1}\left(f(X_t) + R^{(\cdot k)}\right) - X_t
\end{aligned}
$$

for $\beta(X) = \left(D f(X) D f(X) ^T \right) ^{-1} D f(X)$. Thus we obtain by Ito's lemma (note that $D^2 f$ is a 3d-tensor indexed by $i,j,k$ with $k\in 1,\dots, n$)

$$
(\dots)dt = \frac 12 \sum_{i,j=1}^d (\beta(X_t) Z_t(\beta(X_t) Z_t)^T)^{(ij)} (D^2 f(X_t))^{(ij)} dt
$$

Altogether, we can rewrite

$$
\begin{aligned}
dX_t &= \beta(X_t) Z_t dW_t + \left(f^{-1}\left(f(X_t) + R^{(\cdot k)}\right) - X_t\right) dN_t\\
dY_t &= \frac 12 \sum_{i,j=1}^d (\beta(X_t) Z_t(\beta(X_t) Z_t)^T)^{(ij)} (D^2 f(X_t))^{(ij)} dt + Z_t dW_t + R_t dN_t
\end{aligned}
$$

# Example 1

Consider $d=\ell=m=n$ and $f(X) = g(X) = \exp(X)$, elementwise. Then

$$
\begin{aligned}
f^{-1}(X) &= \log(X)\\
Df(X)^{(ij)} &= \delta_{ij} \exp(X^{(i)}) =\operatorname{diag}(\exp(X))\\
D^2 f(X)^{(ijk)} &= \delta_{ijk} \exp(X^{(i)})\\
\beta(X) &= \delta_{ij}\exp(-X^{(i)}) = \operatorname{diag}(\exp(-X))
\end{aligned}
$$

Thus, the system can be rewritten (drift of dY is k-valued vector, $\operatorname{diag}$ means extracting diagonal, or converting vector into diagonal matrix depending on the context):

$$
\begin{aligned}
dX_t &= \operatorname{diag}(\exp(-X)) Z_t dW_t + \left(\log\left(\exp(X_t) + R^{(\cdot k)}\right) - X_t\right) dN_t\\
dY_t &= \frac 12 \operatorname{diag} ( \operatorname{diag}(\exp(-X)) Z_t( \operatorname{diag}(\exp(-X)) Z_t)^T) \exp(X_t) dt + Z_t dW_t + R_t dN_t
\end{aligned}
$$

Thus the solution is given by $Y_t = \exp(X_t)$.

# Example 1a

Consider $d=\ell=m=n$ and $f(X) = g(X) = \exp(aX)$, elementwise. Then

$$
\begin{aligned}
f^{-1}(X) &= \frac 1a \log(X)\\
Df(X)^{(ij)} &= a\delta_{ij} \exp(aX^{(i)}) = a\operatorname{diag}(\exp(aX))\\
D^2 f(X)^{(ijk)} &= a^2 \delta_{ijk} \exp(aX^{(i)})\\
\beta(X) &= \frac 1a \delta_{ij}\exp(-aX^{(i)}) = \frac 1a \operatorname{diag}(\exp(-aX))
\end{aligned}
$$

Thus, the system can be rewritten (drift of dY is k-valued vector, $\operatorname{diag}$ means extracting diagonal, or converting vector into diagonal matrix depending on the context):

$$
\begin{aligned}
dX_t &= \frac 1a \operatorname{diag}(\exp(-aX)) Z_t dW_t + \left(\frac 1a \log\left(\exp(aX_t) + R^{(\cdot k)}\right) - X_t\right) dN_t\\
dY_t &= \frac {1}{2} \operatorname{diag} ( \operatorname{diag}(\exp(-aX)) Z_t( \operatorname{diag}(\exp(-aX)) Z_t)^T) \exp(aX_t) dt + Z_t dW_t + R_t dN_t
\end{aligned}
$$

Thus the solution is given by $Y_t = \exp(X_t)$.

# Implementation

In [1]:
import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, Lambda, Reshape, concatenate, Layer, BatchNormalization
from keras import Model, initializers
from keras.callbacks import ModelCheckpoint
from keras.metrics import mean_squared_error
import matplotlib.pyplot as plt
from datetime import datetime
from keras.metrics import mse
from keras.optimizers import Adam
from keras.callbacks import Callback
import pandas as pd
from tqdm import tqdm
from itertools import product

/usr/lib64/python3.6/site-packages/h5py/__init__.py:39: UserWarning: h5py is running against HDF5 1.10.8 when it was built against 1.10.7, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [3]:
import os
from gpu_utils import *
os.environ["CUDA_VISIBLE_DEVICES"] = str(pick_gpu_lowest_memory())

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num GPUs Available:  1


In [5]:
np.set_printoptions(edgeitems=11, linewidth=90, formatter=dict(float=lambda x: "%7.5g" % x))

In [6]:
# create dirs
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
model_name = f"{timestamp}"
tb_log_dir = "/home/tmp/starokon/tensorboard/" + model_name
output_dir = f"_output/models/{model_name}"
os.makedirs(output_dir, exist_ok=True)

# Inputs

In [7]:
# model parameters
n_paths = 2 ** 8
n_timesteps = 2
time_horizon = 1.
n_x_dimensions = 2
n_y_dimensions = 2
n_diffusion_factors = 2
n_jump_factors = 2
intensity = 1.
alpha = 1.

In [8]:
dt = time_horizon / n_timesteps

In [9]:
def b(t, x, y, z, r):
    return tf.zeros((n_x_dimensions,))

def s(t, x, y, z, r):
    return tf.linalg.matmul(tf.linalg.diag(tf.exp(-alpha * x)), z) / alpha

def v(t, x, y, z, r):
    log_result = tf.math.log(tf.exp(alpha * x) + r) / alpha
    log_result = tf.where(tf.math.is_nan(log_result), 0., log_result)
    return log_result - x

def f(t, x, y, z, r):
    res = tf.linalg.matmul(tf.linalg.diag(tf.exp(-alpha * x)), z)
    return 0.5 * tf.einsum('ij,j', tf.linalg.matmul(res, res, transpose_b=True), tf.exp(alpha * x))

def g(x):
    return tf.exp(x)


# Custom layers and callbacks

In [10]:
class InitialValue(Layer):
    
    def __init__(self, y0, **kwargs):
        super().__init__(**kwargs)
        self.y0 = y0
    
    def call(self, inputs):
        return self.y0


In [11]:
class Y0Callback(Callback):
        
    def on_epoch_end(self, epoch, logs=None):
        y0 = self.model.get_layer('y_0').y0.numpy()
        print(y0)

# Model

In [12]:
def build_model(n_x_dimensions, n_y_dimensions, n_diffusion_factors, n_jump_factors, n_timesteps, time_horizon):

    dt = time_horizon / n_timesteps

    def dX(t, x, y, z, r, dW, dN):

        def drift(arg):
            x, y, z, r = arg
            return tf.math.multiply(b(t, x, y, z, r), dt)
        a0 = tf.vectorized_map(drift, (x, y, z, r))

        def noise(arg):
            x, y, z, r, dW = arg
            return tf.einsum('ij,j', s(t, x, y, z, r), dW)
        a1 = tf.vectorized_map(noise, (x, y, z, r, dW))

        def jump(arg):
            x, y, z, r, dN = arg
            return tf.einsum('ij,j', v(t, x, y, z, r), dN)
        a2 = tf.vectorized_map(jump, (x, y, z, r, dN))

        return a0 + a1 + a2

    def dY(t, x, y, z, r, dW, dN):

        def drift(arg):
            x, y, z, r = arg
            return tf.math.multiply(f(t, x, y, z, r), dt)
        a0 = tf.vectorized_map(drift, (x, y, z, r))

        def noise(arg):
            x, y, z, r, dW = arg
            return tf.einsum('ij,j', z, dW)
        a1 = tf.vectorized_map(noise, (x, y, z, r, dW))

        def jump(arg):
            x, y, z, r, dN = arg
            return tf.einsum('ij,j', r, dN)
        a2 = tf.vectorized_map(jump, (x, y, z, r, dN))        

        return a0 + a1 + a2

    @tf.function
    def hx(args):
        i, x, y, z, r, dW, dN = args
        return x + dX(i * dt, x, y, z, r, dW, dN)

    @tf.function
    def hy(args):
        i, x, y, z, r, dW, dN = args
        return y + dY(i * dt, x, y, z, r, dW, dN)

    paths = []

    n_hidden_units = n_x_dimensions + n_diffusion_factors + n_jump_factors + 10

    inputs_x0 = Input(shape=(n_x_dimensions))
    inputs_dW = Input(shape=(n_timesteps, n_diffusion_factors))
    inputs_dN = Input(shape=(n_timesteps, n_jump_factors))

    # constant x0

    x0 = tf.Variable([[1. for _ in range(n_x_dimensions)]], trainable=False)
    y0 = tf.Variable([[0. for _ in range(n_y_dimensions)]], trainable=True)

    x = InitialValue(x0, trainable=False, name='x_0')(inputs_dW)
    y = InitialValue(y0, trainable=True, name='y_0')(inputs_dW)

    # adjoints

    z = concatenate([x, y])
    z = Dense(n_hidden_units, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-2), name='z1_0')(z)
    z = Dense(n_y_dimensions * n_diffusion_factors, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-2), name='z2_0')(z)
    z = BatchNormalization(name='zbn_0')(z)
    z = Reshape((n_y_dimensions, n_diffusion_factors), name='zr_0')(z)

    r = concatenate([x, y])
    r = Dense(n_hidden_units, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-2), name='r1_0')(r)
    r = Dense(n_y_dimensions * n_jump_factors, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-2), name='r2_0')(r)
    r = BatchNormalization(name='rbn_0')(r)
    r = Reshape((n_y_dimensions, n_jump_factors), name='rr_0')(r)

    paths += [[x, y, z, r]]

    # pre-compile lambda layers

    for i in range(n_timesteps):

        step = InitialValue(tf.Variable(i, dtype=tf.float32, trainable=False))(inputs_dW)

        dW = Lambda(lambda x: x[0][:, tf.cast(x[1], tf.int32)])([inputs_dW, step])
        dN = Lambda(lambda x: x[0][:, tf.cast(x[1], tf.int32)])([inputs_dN, step])

        x, y = (
            Lambda(hx, name=f'x_{i+1}')([step, x, y, z, r, dW, dN]),
            Lambda(hy, name=f'y_{i+1}')([step, x, y, z, r, dW, dN]),
        )

        # we don't train z for the last time step; keep for consistency
        z = concatenate([x, y])
        z = Dense(n_hidden_units, activation='relu', name=f'z1_{i+1}')(z)
        z = Dense(n_y_dimensions * n_diffusion_factors, activation='relu', name=f'z2_{i+1}')(z)
        z = Reshape((n_y_dimensions, n_diffusion_factors), name=f'zr_{i+1}')(z)
        z = BatchNormalization(name=f'zbn_{i+1}')(z)

        # we don't train r for the last time step; keep for consistency
        r = concatenate([x, y])
        r = Dense(n_hidden_units, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-2), name=f'r1_{i+1}')(r)
        r = Dense(n_y_dimensions * n_jump_factors, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-2), name=f'r2_{i+1}')(r)
        r = Reshape((n_y_dimensions, n_jump_factors), name=f'rr_{i+1}')(r)
        r = BatchNormalization(name=f'rbn_{i+1}')(r)

        paths += [[x, y, z, r]]

    outputs_loss = Lambda(lambda r: r[1] - tf.vectorized_map(g, r[0]))([x, y])
    
    # remember that z and r are matrices
    outputs_paths = tf.stack(
        [tf.stack([p[0] for p in paths[1:]], axis=1)] + 
        [tf.stack([p[1] for p in paths[1:]], axis=1)] + 
        [tf.stack([p[2][:, :, i] for p in paths[1:]], axis=1) for i in range(n_diffusion_factors)] +
        [tf.stack([p[3][:, :, i] for p in paths[1:]], axis=1) for i in range(n_jump_factors)], axis=2)

    model_loss = Model([inputs_x0, inputs_dW, inputs_dN], outputs_loss)

    # (n_sample, n_timestep, x/y/z_k, n_dimension)
    # skips the first time step
    model_paths = Model([inputs_x0, inputs_dW, inputs_dN], outputs_paths)

    return model_loss, model_paths


In [13]:
# create model
dt = time_horizon / n_timesteps
model_loss, model_paths = build_model(n_x_dimensions=n_x_dimensions,
                                      n_y_dimensions=n_y_dimensions,
                                      n_diffusion_factors=n_diffusion_factors,
                                      n_jump_factors=n_jump_factors,
                                      n_timesteps=n_timesteps,
                                      time_horizon=time_horizon)

# Training

In [14]:
x0 = tf.constant(np.full((n_paths, n_x_dimensions), 1.), dtype=tf.float32)
dW = tf.sqrt(dt) * tf.random.normal((n_paths, n_timesteps, n_diffusion_factors))
dN = tf.random.poisson((n_paths, n_timesteps), tf.constant(dt * np.array([intensity for _ in range(n_jump_factors)])))
target = tf.zeros((n_paths, n_y_dimensions))

In [15]:
# define callbacks
callbacks = []

callbacks += [Y0Callback()]
callbacks += [ModelCheckpoint(os.path.join(output_dir, "model.h5"), monitor="loss", save_weights_only=True, save_best_only=True, overwrite=True)]
# callbacks += [tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1, profile_batch=1)]
callbacks += [tf.keras.callbacks.TerminateOnNaN()]
callbacks += [tf.keras.callbacks.EarlyStopping(monitor="loss", min_delta=1e-4, patience=30)]


In [18]:
def mse_loss(y_true, y_pred):
    print("\n", y_pred.numpy().transpose())
    return(tf.keras.metrics.mean_squared_error(y_true, y_pred))

adam = Adam(learning_rate=1e-2) 
model_loss.compile(loss=mse_loss, optimizer=adam, run_eagerly=True)

In [19]:
history = model_loss.fit([x0, dW, dN], target, batch_size=8, initial_epoch=0, epochs=1000, callbacks=callbacks)
df_loss = pd.DataFrame(history.history['loss'])
df_loss.to_csv(os.path.join(output_dir, 'loss.csv'))

Epoch 1/1000

 [[-3.6715 -3.7782 -2.7185 -2.7235  -2.792 -2.8935 -3.1528 -3.0259]
 [-3.3016 -3.3519 -2.7185 -2.7233 -2.8049 -2.8581  -3.026 -2.9475]]
 1/32 [..............................] - ETA: 27s - loss: 9.2906
 [[-3.2444 -3.3883 -2.9969 -2.8031 -2.7319 -2.7241 -2.9479 -2.8231]
 [-3.5273 -3.8782 -3.0989 -2.8115 -2.7211 -2.7179 -2.8947 -2.8474]]
 2/32 [>.............................] - ETA: 4s - loss: 9.2284 
 [[-2.7069 -2.7024 -2.7739 -3.1591 -2.7167 -2.7728 -2.7418 -2.7178]
 [-2.6978 -2.6989 -2.7619 -3.0314 -2.7105 -2.7804  -2.733 -2.7075]]
 3/32 [=>............................] - ETA: 4s - loss: 8.7258
 [[-2.7643 -2.6903  -3.129 -2.8051 -2.8203 -2.7156 -2.6818 -2.6773]
 [-2.7296 -2.7191 -3.0319 -2.8165 -2.6994 -2.7121 -2.6571 -2.6837]]
 4/32 [==>...........................] - ETA: 4s - loss: 8.4678WARNING:tensorflow:5 out of the last 5 calls to <function pfor.<locals>.f at 0x7f23d46b9268> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings c


 [[-2.7413 -2.8654 -2.6856 -2.9981 -2.7091 -2.6714 -2.9525 -2.6737]
 [-2.7623 -2.8963 -2.6882 -2.9553 -2.6624 -2.6436 -2.9059 -2.6245]]
 5/32 [===>..........................] - ETA: 3s - loss: 8.3200WARNING:tensorflow:6 out of the last 6 calls to <function pfor.<locals>.f at 0x7f23d46b9620> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.



 [[-2.6801 -2.6795 -3.3816 -2.8454 -3.0496 -2.7678 -2.7976 -3.2149]
 [-2.7438 -2.6685 -3.2306 -2.9006 -3.1667 -2.7964 -2.7725 -3.4979]]
 6/32 [====>.........................] - ETA: 3s - loss: 8.3946
 [[ -2.665 -3.1856 -2.6692 -4.1343 -2.6926 -3.0854 -2.6601 -2.6988]
 [-2.6525 -3.0706 -2.6484 -3.7551 -2.8555 -2.9775 -2.6594 -2.6861]]
 7/32 [=====>........................] - ETA: 3s - loss: 8.4586
 [[-2.8573 -2.6591 -2.6595 -2.7303 -2.6507 -2.5982 -2.7648 -2.6712]
 [-2.9141 -2.6595 -2.6426 -2.6515 -2.6426 -2.5192 -2.7439 -2.6659]]
 8/32 [======>.......................] - ETA: 3s - loss: 8.3065
 [[-2.6456 -2.6584 -2.7451  -2.697 -2.7131 -2.6267 -2.7514 -2.5927]
 [-2.6344 -2.6753  -2.849 -2.7106 -2.7135 -2.6099 -2.7051 -2.5754]]
 9/32 [=======>......................] - ETA: 3s - loss: 8.1830
 [[-2.6389 -2.7931 -2.8813 -2.7218 -2.6375   -2.64 -2.6351 -2.6339]
 [-2.6396 -2.7749  -2.985 -2.7296 -2.6219 -2.6697 -2.6304 -2.6304]]
10/32 [========>.....................] - ETA: 3s - loss: 8.0969